In [76]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [78]:
# 데이터 로드
data = pd.read_csv("C:\\Users\\LG\\Downloads\\BP_data.csv")

In [80]:
data.isnull().sum()

Patient_Number                      0
Blood_Pressure_Abnormality          0
Level_of_Hemoglobin                 0
Genetic_Pedigree_Coefficient       92
Age                                 0
BMI                                 0
Sex                                 0
Pregnancy                        1558
Smoking                             0
Physical_activity                   0
salt_content_in_the_diet            0
alcohol_consumption_per_day       242
Level_of_Stress                     0
Chronic_kidney_disease              0
Adrenal_and_thyroid_disorders       0
dtype: int64

In [82]:
data = data.fillna(0)

In [84]:
# 불필요 컬럼 제거, 레이블을 예측 타겟으로 설정
X = data.drop(columns=['Patient_Number', 'Blood_Pressure_Abnormality']).values  
y = data['Blood_Pressure_Abnormality'].values.astype(np.float32)         

In [86]:
# 정규화
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [88]:
# 학습/테스트 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [90]:
# 텐서로 변환
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

In [92]:
# DataLoader 구성
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [94]:
# MLP 회귀 모델 정의
class BPRegressionModel(nn.Module):
    def __init__(self):
        super(BPRegressionModel, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(13, 64),     # 입력 차원: 13
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)      
        )

    def forward(self, x):
        return self.model(x)

In [96]:
# 학습 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BPRegressionModel().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [98]:
# 학습 루프
model.train()
for epoch in range(50):
    total_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")

Epoch 1, Loss: 0.3654
Epoch 2, Loss: 0.1847
Epoch 3, Loss: 0.1672
Epoch 4, Loss: 0.1592
Epoch 5, Loss: 0.1527
Epoch 6, Loss: 0.1447
Epoch 7, Loss: 0.1376
Epoch 8, Loss: 0.1317
Epoch 9, Loss: 0.1256
Epoch 10, Loss: 0.1195
Epoch 11, Loss: 0.1131
Epoch 12, Loss: 0.1105
Epoch 13, Loss: 0.1068
Epoch 14, Loss: 0.1050
Epoch 15, Loss: 0.1030
Epoch 16, Loss: 0.1003
Epoch 17, Loss: 0.0987
Epoch 18, Loss: 0.0970
Epoch 19, Loss: 0.0940
Epoch 20, Loss: 0.0926
Epoch 21, Loss: 0.0921
Epoch 22, Loss: 0.0905
Epoch 23, Loss: 0.0877
Epoch 24, Loss: 0.0869
Epoch 25, Loss: 0.0842
Epoch 26, Loss: 0.0825
Epoch 27, Loss: 0.0807
Epoch 28, Loss: 0.0793
Epoch 29, Loss: 0.0770
Epoch 30, Loss: 0.0762
Epoch 31, Loss: 0.0737
Epoch 32, Loss: 0.0722
Epoch 33, Loss: 0.0707
Epoch 34, Loss: 0.0701
Epoch 35, Loss: 0.0692
Epoch 36, Loss: 0.0674
Epoch 37, Loss: 0.0672
Epoch 38, Loss: 0.0661
Epoch 39, Loss: 0.0632
Epoch 40, Loss: 0.0623
Epoch 41, Loss: 0.0601
Epoch 42, Loss: 0.0604
Epoch 43, Loss: 0.0582
Epoch 44, Loss: 0.05

In [99]:
# 평가
model.eval()
preds, actuals = [], []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        output = model(X_batch).cpu().numpy()
        preds.extend(output)
        actuals.extend(y_batch.numpy())

mse = mean_squared_error(actuals, preds)
print(f"Test MSE: {mse:.4f}")

Test MSE: 0.1503
